In [1]:
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import OutputType
import pandas as pd
import datetime as datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from pulp import *
import numpy as np
from sklearn import linear_model

In [2]:
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)

In [3]:
client.player_box_scores(day=1, month=1, year=2020)

[{'slug': 'jamesle01',
  'name': 'LeBron James',
  'team': <Team.LOS_ANGELES_LAKERS: 'LOS ANGELES LAKERS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.PHOENIX_SUNS: 'PHOENIX SUNS'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 2284,
  'made_field_goals': 11,
  'attempted_field_goals': 21,
  'made_three_point_field_goals': 1,
  'attempted_three_point_field_goals': 4,
  'made_free_throws': 8,
  'attempted_free_throws': 12,
  'offensive_rebounds': 2,
  'defensive_rebounds': 11,
  'assists': 12,
  'steals': 2,
  'blocks': 1,
  'turnovers': 5,
  'personal_fouls': 0,
  'game_score': 29.9},
 {'slug': 'antetgi01',
  'name': 'Giannis Antetokounmpo',
  'team': <Team.MILWAUKEE_BUCKS: 'MILWAUKEE BUCKS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 1889,
  'made_field_goals': 13,
  'attempted_field_goals': 22,
  'made_three_point_field_goals': 3,
  'atte

In [2]:
all_tables = []
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

In [3]:
full_df = all_tables[0]
for i in range(1, len(all_tables)):
    current_table = all_tables[i]
    full_df = full_df.append(current_table)
full_df.to_csv("./OutputCSVs/all_games.csv")

In [4]:
df = pd.read_csv("./OutputCSVs/teams_stats.csv")
df["Team"] = df["Team"].str.upper()
all_games = pd.read_csv("./OutputCSVs/all_games.csv")
x = all_games[["team", "opponent"]].loc[0]["team"]
df[df["Team"] == x]["TOV%"]

team_def = []
team_pace = []
team_tov = []
opp_def = []
opp_pace = []
opp_tov = []
all_games_teams = all_games[["team", "opponent"]]

for i in range(len(all_games_teams)):
    game = all_games_teams.loc[i]
    team = game["team"]
    opponent = game["opponent"]
    team_def.append(df[df["Team"] == team]["DRtg"].iloc[0])
    team_pace.append(df[df["Team"] == team]["Pace"].iloc[0])
    team_tov.append(df[df["Team"] == team]["TOV%"].iloc[0])
    opp_def.append(df[df["Team"] == opponent]["DRtg"].iloc[0])
    opp_pace.append(df[df["Team"] == opponent]["Pace"].iloc[0])
    opp_tov.append(df[df["Team"] == opponent]["TOV%"].iloc[0])
    
all_games["Team Defensive Rating"] = team_def
all_games["Team Pace"] = team_pace
all_games["Team Turnover %"] = team_tov
all_games["Opponent Defensive Rating"] = opp_def
all_games["Opponent Pace"] = opp_pace
all_games["Opponent Turnover %"] = opp_tov

all_games.to_csv("./OutputCSVs/all_games_new.csv")
df = pd.read_csv("./OutputCSVs/all_games_new.csv")
attempted_2s = df["attempted_field_goals"] - df["attempted_three_point_field_goals"]
made_2s = df["made_field_goals"] - df["made_three_point_field_goals"]
rebounds = df["offensive_rebounds"] + df["defensive_rebounds"]
at_home = df["location"] == "HOME"
df["attempted_two_point_field_goals"] = attempted_2s
df["made_two_point_field_goals"] = made_2s
df["total_rebounds"] = rebounds
df["at_home"] = at_home
df.to_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual = pd.read_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual =  all_games_actual.iloc[:, 3:].reset_index()

In [5]:
def get_stats(player, date, number_rows):
    converted_datetime = datetime.strptime(date, '%Y-%m-%d')
    player_rows = all_games_actual.loc[all_games_actual['name'] == player]
    #print(player_rows)
    selected_rows = []
    if (len(player_rows)) < number_rows:
        for i in range(len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
        return pd.DataFrame(selected_rows)
    index = 0
    for i in range(len(player_rows)):
        curr_date = player_rows.iloc[i]['Date']
        if datetime.strptime(curr_date, '%Y-%m-%d') >= converted_datetime:
            index = i - 1
            break
    if index != 0:
        if index + 1 - number_rows < 0:
            for i in range(index + 1):
                selected_rows.append(player_rows.iloc[i])
        else:
            for i in range(index + 1 - number_rows, index + 1):
                selected_rows.append(player_rows.iloc[i])
#     elif converted_datetime >= players_rows[len(player_rows) - 1]:
    else:
        for i in range(len(player_rows) - number_rows, len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
    return pd.DataFrame(selected_rows)

In [6]:
def get_points(row_data):
    three_pt_fgs = row_data['made_three_point_field_goals']
    two_pt_fgs = row_data['made_two_point_field_goals']
    made_fts = row_data['made_free_throws']
    total_rebounds = row_data['rebounds']
    assists = row_data['assists']
    blocks = row_data['blocks']
    steals = row_data['steals']
    turnovers = row_data['turnovers']
    FD_points = three_pt_fgs * 3 + two_pt_fgs * 2 + made_fts + total_rebounds * 1.2 + assists * 1.5 + blocks * 3 + steals * 3 - turnovers
    FD_dollars = FD_points * 200
    return (FD_points, FD_dollars)

In [7]:
def weight_function(statistic, weight):
    s = 0
    if type(statistic) == np.ndarray:
        for i in range(len(statistic)):
            s += statistic[len(statistic) - i - 1] * (weight ** i)
    else:
        for i in range(len(statistic)):
            s += statistic.iloc[len(statistic) - i - 1,] * (weight ** i)
    return s

In [8]:
def time_weighted_average(rows, statistic, weight):
    if rows.empty:
        return 0
    stat = rows[statistic]
    this_num = 1 / weight_function(np.ones(len(stat)), weight)
    return this_num * weight_function(stat, weight)
player_box_scores = pd.read_csv('./OutputCSVs/all_games_updated.csv')

input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals", "seconds_played"]

In [9]:
def statline_predictor(player_box_scores, input_statistics, sample_size = 5, weight = .8):
    player_box_scores = player_box_scores[~player_box_scores.index.duplicated()]
    player_box_scores.reindex(range(len(player_box_scores)), axis = "index")
    predicted_statlines = pd.DataFrame(index = player_box_scores.index, columns = input_statistics).fillna(0).T
    for box_index in player_box_scores.index:
        box_score = player_box_scores.loc[box_index]
        player_name = box_score["name"]
        game_date = str(box_score["Date"])[:10]
        print(player_name)
        print(box_index)
        print(game_date)
        last_n_rows = get_stats(player_name, game_date, sample_size)
        weighted_stats = [player_name, box_score["team"], game_date, box_score["location"], box_score["opponent"]]
        for stat in input_statistics[5:]:
            weighted_stats.append(round(time_weighted_average(last_n_rows, stat, weight), 2))
        predicted_statlines[box_index] = weighted_stats
    return predicted_statlines.T

In [10]:
predictions_samplesize5 = statline_predictor(all_games_actual, input_statistics, sample_size = 5)
predictions_samplesize5.to_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
predictions_samplesize10 = statline_predictor(all_games_actual, input_statistics, sample_size = 10)
predictions_samplesize10.to_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")

LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
Nikola Vučević
16
2020-01-01
Bobby Portis
17
2020-01-01
Kyle Kuzma
18
2020-01-01
Khem Birch
19
2020-01-01
Robert Covington
20
2020-01-01
Marcus Morris
21
2020-01-01
Terrence Ross
22
2020-01-01
CJ McCollum
23
2020-01-01
Donte DiVincenzo
24
2020-01-01
Jordan McRae
25
2020-01-01
Troy Brown
26
2020-01-01
Josh Okogie
27
2020-01-01
Ricky Rubio
28
2020-01-01
Wesley Iwundu
29
2020-01-01
Ish Smith
30
2020-01-01
Gorgui Dieng
31
2020-01-01
Brook Lopez
32
2020-01-01
Eric Bledsoe
33
2020-01-01
Evan Fournier
34
2020-01-01
Ersan İlyasova
35
2020-01-01
Gar

Derrick Favors
286
2020-01-03
LeBron James
287
2020-01-03
CJ McCollum
288
2020-01-03
Nikola Vučević
289
2020-01-03
Marcus Smart
290
2020-01-03
Jaylen Brown
291
2020-01-03
Trae Young
292
2020-01-03
Bobby Portis
293
2020-01-03
Tobias Harris
294
2020-01-03
Bam Adebayo
295
2020-01-03
Deandre Ayton
296
2020-01-03
Aron Baynes
297
2020-01-03
Joel Embiid
298
2020-01-03
Kevin Huerter
299
2020-01-03
Garrison Mathews
300
2020-01-03
Carmelo Anthony
301
2020-01-03
Enes Kanter
302
2020-01-03
Gordon Hayward
303
2020-01-03
Daniel Theis
304
2020-01-03
John Collins
305
2020-01-03
J.J. Redick
306
2020-01-03
Evan Fournier
307
2020-01-03
Aaron Gordon
308
2020-01-03
Elfrid Payton
309
2020-01-03
Ricky Rubio
310
2020-01-03
Ish Smith
311
2020-01-03
E'Twaun Moore
312
2020-01-03
Russell Westbrook
313
2020-01-03
Johnathan Williams
314
2020-01-03
Trey Burke
315
2020-01-03
Brandon Goodwin
316
2020-01-03
Markelle Fultz
317
2020-01-03
Mitchell Robinson
318
2020-01-03
Brandon Ingram
319
2020-01-03
Jrue Holiday
320
202

Dean Wade
580
2020-01-04
Malik Monk
581
2020-01-04
Gary Payton
582
2020-01-04
Derrick Walton
583
2020-01-04
Juan Hernangómez
584
2020-01-04
Jahlil Okafor
585
2020-01-04
Rayjon Tucker
586
2020-01-04
Dwayne Bacon
587
2020-01-04
Keldon Johnson
588
2020-01-04
Vlatko Čančar
589
2020-01-04
Jordan McRae
590
2020-01-04
Džanan Musa
591
2020-01-04
Tristan Thompson
592
2020-01-04
Marko Guduric
593
2020-01-04
Isaiah Thomas
594
2020-01-04
Delon Wright
595
2020-01-04
Jalen Brunson
596
2020-01-04
Matthew Dellavedova
597
2020-01-04
Ante Žižić
598
2020-01-04
Daniel Gafford
599
2020-01-04
Thon Maker
600
2020-01-04
Coby White
601
2020-01-04
Justin James
602
2020-01-04
Amile Jefferson
603
2020-01-04
Cody Martin
604
2020-01-04
Malcolm Miller
605
2020-01-04
Juwan Morgan
606
2020-01-04
Josh Reaves
607
2020-01-04
Rodney McGruder
608
2020-01-04
Boban Marjanović
609
2020-01-04
Semi Ojeleye
610
2020-01-04
Theo Pinson
611
2020-01-04
Edmond Sumner
612
2020-01-04
D.J. Wilson
613
2020-01-04
Torrey Craig
614
2020-01-

Mason Plumlee
875
2020-01-06
Harry Giles
876
2020-01-06
Nickeil Alexander-Walker
877
2020-01-06
Kris Dunn
878
2020-01-06
Brandon Goodwin
879
2020-01-06
Edmond Sumner
880
2020-01-06
Robin Lopez
881
2020-01-06
Mohamed Bamba
882
2020-01-06
Chimezie Metu
883
2020-01-06
P.J. Washington
884
2020-01-06
Mike Muscala
885
2020-01-06
Jakob Poeltl
886
2020-01-06
Justin Anderson
887
2020-01-06
Garrison Mathews
888
2020-01-06
Alen Smailagić
889
2020-01-06
Garrett Temple
890
2020-01-06
Ersan İlyasova
891
2020-01-06
Javonte Green
892
2020-01-06
DeAndre' Bembry
893
2020-01-06
Malik Monk
894
2020-01-06
Jacob Evans
895
2020-01-06
Kenrich Williams
896
2020-01-06
Caleb Swanigan
897
2020-01-06
JaKarr Sampson
898
2020-01-06
Ky Bowman
899
2020-01-06
Vince Carter
900
2020-01-06
Seth Curry
901
2020-01-06
Shaquille Harrison
902
2020-01-06
Paul Millsap
903
2020-01-06
Bismack Biyombo
904
2020-01-06
Thanasis Antetokounmpo
905
2020-01-06
DeMarre Carroll
906
2020-01-06
James Ennis
907
2020-01-06
Chandler Hutchison
90

Jaylen Brown
1157
2020-01-08
Malik Monk
1158
2020-01-08
Jamal Murray
1159
2020-01-08
Jordan Clarkson
1160
2020-01-08
Jakob Poeltl
1161
2020-01-08
Meyers Leonard
1162
2020-01-08
Coby White
1163
2020-01-08
Elfrid Payton
1164
2020-01-08
Alen Smailagić
1165
2020-01-08
Jayson Tatum
1166
2020-01-08
Malik Beasley
1167
2020-01-08
Dorian Finney-Smith
1168
2020-01-08
Brandon Goodwin
1169
2020-01-08
Draymond Green
1170
2020-01-08
Mitchell Robinson
1171
2020-01-08
Gary Harris
1172
2020-01-08
Matt Thomas
1173
2020-01-08
Javonte Green
1174
2020-01-08
Mohamed Bamba
1175
2020-01-08
Melvin Frazier
1176
2020-01-08
Bismack Biyombo
1177
2020-01-08
Devonte' Graham
1178
2020-01-08
T.J. McConnell
1179
2020-01-08
Kemba Walker
1180
2020-01-08
Ky Bowman
1181
2020-01-08
Isaac Bonga
1182
2020-01-08
Jordan McRae
1183
2020-01-08
Kyle Korver
1184
2020-01-08
Ignas Brazdeikis
1185
2020-01-08
Torrey Craig
1186
2020-01-08
Robin Lopez
1187
2020-01-08
Gary Payton
1188
2020-01-08
Grant Williams
1189
2020-01-08
Chris Bouche

...

Josh Richardson
6734
2020-02-20
Cody Martin
6735
2020-02-20
Bruno Fernando
6736
2020-02-20
Shake Milton
6737
2020-02-20
Kevon Looney
6738
2020-02-20
John Henson
6739
2020-02-20
Luke Kornet
6740
2020-02-20
Sviatoslav Mykhailiuk
6741
2020-02-20
Jabari Parker
6742
2020-02-20
Furkan Korkmaz
6743
2020-02-20
Timothé Luwawu-Cabarrot
6744
2020-02-20
Daniel Gafford
6745
2020-02-20
Solomon Hill
6746
2020-02-20
Ersan İlyasova
6747
2020-02-20
Thabo Sefolosha
6748
2020-02-20
Taurean Waller-Prince
6749
2020-02-20
Marvin Williams
6750
2020-02-20
Harry Giles
6751
2020-02-20
Jalen McDaniels
6752
2020-02-20
Marko Guduric
6753
2020-02-20
Ryan Arcidiacono
6754
2020-02-20
D.J. Wilson
6755
2020-02-20
Kelly Olynyk
6756
2020-02-20
Brandon Goodwin
6757
2020-02-20
Michael Frazier
6758
2020-02-20
Bismack Biyombo
6759
2020-02-20
Justin James
6760
2020-02-20
Bruno Caboclo
6761
2020-02-20
Pat Connaughton
6762
2020-02-20
Vince Carter
6763
2020-02-20
Matisse Thybulle
6764
2020-02-20
Dewayne Dedmon
6765
2020-02-20
Gor

Daniel Gafford
7009
2020-02-22
Collin Sexton
7010
2020-02-22
Cody Zeller
7011
2020-02-22
Joe Harris
7012
2020-02-22
Pat Connaughton
7013
2020-02-22
Mike Conley
7014
2020-02-22
Joel Embiid
7015
2020-02-22
Rudy Gobert
7016
2020-02-22
Kelly Oubre
7017
2020-02-22
Spencer Dinwiddie
7018
2020-02-22
DeAndre Jordan
7019
2020-02-22
Maxi Kleber
7020
2020-02-22
Taurean Waller-Prince
7021
2020-02-22
Eric Bledsoe
7022
2020-02-22
Justin Jackson
7023
2020-02-22
Shaquille Harrison
7024
2020-02-22
Zach LaVine
7025
2020-02-22
Brook Lopez
7026
2020-02-22
Nemanja Bjelica
7027
2020-02-22
Dewayne Dedmon
7028
2020-02-22
Tobias Harris
7029
2020-02-22
Robin Lopez
7030
2020-02-22
Eric Gordon
7031
2020-02-22
Glenn Robinson
7032
2020-02-22
Cristiano Felício
7033
2020-02-22
Bruno Fernando
7034
2020-02-22
Marvin Williams
7035
2020-02-22
Cameron Johnson
7036
2020-02-22
Larry Nance
7037
2020-02-22
Royce O'Neale
7038
2020-02-22
Malik Monk
7039
2020-02-22
Wesley Matthews
7040
2020-02-22
Terry Rozier
7041
2020-02-22
Rya

Mitchell Robinson
7290
2020-02-24
Terrence Ross
7291
2020-02-24
Kendrick Nunn
7292
2020-02-24
Spencer Dinwiddie
7293
2020-02-24
Larry Nance
7294
2020-02-24
Gorgui Dieng
7295
2020-02-24
Darius Garland
7296
2020-02-24
Evan Fournier
7297
2020-02-24
James Johnson
7298
2020-02-24
Kevin Love
7299
2020-02-24
Delon Wright
7300
2020-02-24
De'Andre Hunter
7301
2020-02-24
Deandre Ayton
7302
2020-02-24
Nikola Vučević
7303
2020-02-24
Rudy Gobert
7304
2020-02-24
Landry Shamet
7305
2020-02-24
Seth Curry
7306
2020-02-24
Kelan Martin
7307
2020-02-24
Tristan Thompson
7308
2020-02-24
Derrick Jones
7309
2020-02-24
John Konchar
7310
2020-02-24
Giannis Antetokounmpo
7311
2020-02-24
Kelly Oubre
7312
2020-02-24
Julius Randle
7313
2020-02-24
James Ennis
7314
2020-02-24
Ish Smith
7315
2020-02-24
Danuel House
7316
2020-02-24
JaMychal Green
7317
2020-02-24
Furkan Korkmaz
7318
2020-02-24
Robin Lopez
7319
2020-02-24
Dario Šarić
7320
2020-02-24
Jeff Green
7321
2020-02-24
Kristaps Porziņģis
7322
2020-02-24
Bojan Bogd

Vincent Poirier
7566
2020-02-25
Robin Lopez
7567
2020-02-25
E'Twaun Moore
7568
2020-02-25
Carsen Edwards
7569
2020-02-25
Jahlil Okafor
7570
2020-02-25
Nassir Little
7571
2020-02-25
Grant Williams
7572
2020-02-25
Rajon Rondo
7573
2020-02-25
Sekou Doumbouya
7574
2020-02-25
Khyri Thomas
7575
2020-02-25
Luguentz Dort
7576
2020-02-25
Nemanja Bjelica
7577
2020-02-25
Romeo Langford
7578
2020-02-25
Dragan Bender
7579
2020-02-25
Avery Bradley
7580
2020-02-25
Serge Ibaka
7581
2020-02-25
Caris LeVert
7582
2020-02-26
Trae Young
7583
2020-02-26
Aaron Gordon
7584
2020-02-26
Russell Westbrook
7585
2020-02-26
DeMar DeRozan
7586
2020-02-26
Jayson Tatum
7587
2020-02-26
Kristaps Porziņģis
7588
2020-02-26
Evan Fournier
7589
2020-02-26
John Collins
7590
2020-02-26
D'Angelo Russell
7591
2020-02-26
Kawhi Leonard
7592
2020-02-26
Bradley Beal
7593
2020-02-26
Donovan Mitchell
7594
2020-02-26
Austin Rivers
7595
2020-02-26
James Harden
7596
2020-02-26
Collin Sexton
7597
2020-02-26
Terry Rozier
7598
2020-02-26
Mar

Doug McDermott
7839
2020-02-27
Anfernee Simons
7840
2020-02-27
Wenyen Gabriel
7841
2020-02-27
Kevin Knox
7842
2020-02-27
Terrance Ferguson
7843
2020-02-27
Yogi Ferrell
7844
2020-02-27
Bradley Beal
7845
2020-02-28
Seth Curry
7846
2020-02-28
John Collins
7847
2020-02-28
Giannis Antetokounmpo
7848
2020-02-28
Terrence Ross
7849
2020-02-28
Brandon Ingram
7850
2020-02-28
Derrick Rose
7851
2020-02-28
Spencer Dinwiddie
7852
2020-02-28
Nikola Vučević
7853
2020-02-28
Christian Wood
7854
2020-02-28
Juan Hernangómez
7855
2020-02-28
Cam Reddish
7856
2020-02-28
Montrezl Harrell
7857
2020-02-28
Collin Sexton
7858
2020-02-28
Aaron Gordon
7859
2020-02-28
Duncan Robinson
7860
2020-02-28
OG Anunoby
7861
2020-02-28
Ricky Rubio
7862
2020-02-28
De'Aaron Fox
7863
2020-02-28
Donovan Mitchell
7864
2020-02-28
Deandre Ayton
7865
2020-02-28
Dillon Brooks
7866
2020-02-28
Jrue Holiday
7867
2020-02-28
D'Angelo Russell
7868
2020-02-28
Trae Young
7869
2020-02-28
Jimmy Butler
7870
2020-02-28
Kyle Lowry
7871
2020-02-28


In [20]:
def rebound_predictor_5(weighted_stats):
    if "offensive_rebounds_y" in weighted_stats.index:
        return 3.495 + .9489 * weighted_stats["offensive_rebounds_y"] + .2874 * weighted_stats["defensive_rebounds_y"]
    else:
        return 3.495 + .9489 * weighted_stats["offensive_rebounds"] + .2874 * weighted_stats["defensive_rebounds"]
def freethrow_predictor_5(weighted_stats):
    if "attempted_free_throws_y" in weighted_stats.index:
        return 1.373 + .6217 * weighted_stats["attempted_free_throws_y"] + -.0373 * weighted_stats["attempted_field_goals_y"] -.3289 * (weighted_stats["location_y"] == "HOME")
    else:
        return 1.373 + .6217 * weighted_stats["attempted_free_throws"] + -.0373 * weighted_stats["attempted_field_goals"] -.3289 * (weighted_stats["location"] == "HOME")
def twopt_predictor_5(weighted_stats):
    if "attempted_field_goals_y" in weighted_stats.index:
        return max(0, -5.395 + .6064 * weighted_stats["attempted_two_point_field_goals_y"] - .2542 * weighted_stats["attempted_field_goals_y"] + .071 * weighted_stats["Opponent Defensive Rating"])
    else:
        return max(0, -5.395 + .6064 * weighted_stats["attempted_two_point_field_goals"] - .2542 * weighted_stats["attempted_field_goals"] + .071 * weighted_stats["Opponent Defensive Rating"])
def threept_predictor_5(weighted_stats):
    if "attempted_three_point_field_goals_y" in weighted_stats.index:
        return .766 + .214 * weighted_stats["attempted_three_point_field_goals_y"]
    else:
        return .766 + .214 * weighted_stats["attempted_three_point_field_goals"]
def assist_predictor_5(weighted_stats):
    if "assists_y" in weighted_stats.index:
        return 2.06 + .532 * weighted_stats["assists_y"] - .064 * weighted_stats["blocks_y"]
    else:
        return 2.06 + .532 * weighted_stats["assists"] - .064 * weighted_stats["blocks"]
def block_predictor_5(weighted_stats):
    if "offensive_rebounds_y" in weighted_stats.index:
        return .329 + .182 * weighted_stats["offensive_rebounds_y"] - .038 * weighted_stats["defensive_rebounds_y"] + .424 * weighted_stats["blocks_y"]
    else:
        return .329 + .182 * weighted_stats["offensive_rebounds"] - .038 * weighted_stats["defensive_rebounds"] + .424 * weighted_stats["blocks"]
def steal_predictor_5(weighted_stats):
    if "steals_y" in weighted_stats.index:
        return max(0, -.936 + .339 * weighted_stats["steals_y"] + .129 * weighted_stats["Opponent Turnover %"])
    else:
        return max(0, -.936 + .339 * weighted_stats["steals"] + .129 * weighted_stats["Opponent Turnover %"])
def turnover_predictor_5(weighted_stats):
    if "turnovers_y" in weighted_stats.index:
        return max(0, -2.589 + .18 * weighted_stats["turnovers_y"] + .121 * weighted_stats["assists_y"] + .032 * weighted_stats["Opponent Defensive Rating"])
    else:
        return max(0, -2.589 + .18 * weighted_stats["turnovers"] + .121 * weighted_stats["assists"] + .032 * weighted_stats["Opponent Defensive Rating"])
def minutes_predictor(weighted_stats):
    if "seconds_played_y" in weighted_stats.index:
        return np.mean(weighted_stats["seconds_played_y"])/60
    else:
        return np.mean(weighted_stats["seconds_played"])/60
def recent_average(weighted_stats):
    if "made_three_point_field_goals_y" in weighted_stats.index:
        return 3*np.mean(weighted_stats["made_three_point_field_goals_y"]) + 2*np.mean(weighted_stats["made_two_point_field_goals_y"]) + np.mean(weighted_stats["made_free_throws_y"]) + 1.2*(np.mean(weighted_stats["offensive_rebounds_y"]) + np.mean(weighted_stats["defensive_rebounds_y"])) + 1.5*np.mean(weighted_stats["assists_y"]) + 3*np.mean(weighted_stats["blocks_y"]) + 3*np.mean(weighted_stats["steals_y"]) - np.mean(weighted_stats["turnovers_y"])
    else:
        return 3*np.mean(weighted_stats["made_three_point_field_goals"]) + 2*np.mean(weighted_stats["made_two_point_field_goals"]) + np.mean(weighted_stats["made_free_throws"]) + 1.2*(np.mean(weighted_stats["offensive_rebounds"]) + np.mean(weighted_stats["defensive_rebounds"])) + 1.5*np.mean(weighted_stats["assists"]) + 3*np.mean(weighted_stats["blocks"]) + 3*np.mean(weighted_stats["steals"]) - np.mean(weighted_stats["turnovers"])

In [21]:
all_games_predicted_5 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
all_games_predicted_5 = all_games_predicted_5.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_5 = all_games_actual.merge(all_games_predicted_5, left_on="index", right_on="indexr")
all_games_predicted_10 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")
all_games_predicted_10 = all_games_predicted_10.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_10 = all_games_actual.merge(all_games_predicted_10, left_on="index", right_on="indexr")
all_games_merged_5.to_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")
all_games_merged_10.to_csv("./OutputCSVs/true_and_predicted_statlines_10.csv")

In [22]:
weighted_preds = pd.read_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")

In [73]:
output_statistics = ["name", "team", "date", "location", "opponent", "minutes", "made_two_point_field_goals", "made_three_point_field_goals", "made_free_throws", "rebounds", "assists", "blocks", "steals", "turnovers", "recent_average"]

def statline_output(weighted_predicted_lines):
    output_statlines = pd.DataFrame(index = weighted_predicted_lines.index, columns = output_statistics).fillna(0).T
    for box_index in weighted_predicted_lines.index:
        weighted_line = weighted_predicted_lines.loc[box_index]
        if "name_x" in weighted_line.index:
            output_stats = [weighted_line["name_x"], weighted_line["team_x"], weighted_line["Date"], weighted_line["location_x"], weighted_line["opponent_x"]]
        else:
            output_stats = [weighted_line["name"], weighted_line["team"], weighted_line["date"], weighted_line["location"], weighted_line["opponent"]]
        output_stats.append(minutes_predictor(weighted_line))
        output_stats.append(twopt_predictor_5(weighted_line))
        output_stats.append(threept_predictor_5(weighted_line))
        output_stats.append(freethrow_predictor_5(weighted_line))
        output_stats.append(rebound_predictor_5(weighted_line))
        output_stats.append(assist_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_stats.append(steal_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_stats.append(recent_average(weighted_line))
        output_statlines[box_index] = output_stats
    return output_statlines.T
output_picks = statline_output(weighted_preds)
full_test_output = output_picks[output_picks["date"] > "2020-01-01"]
full_test_output.to_csv("./OutputCSVs/predicted_statlines_jan_feb_2020.csv")

In [17]:
jan_feb_data = pd.read_csv('./OutputCSVs/predicted_statlines_jan_feb_2020.csv')
FD_points_array = []
FD_dollars_array = []
sum_points = 0
counter = 0
for index, row in jan_feb_data.iterrows():
    tuple_values = get_points(row)
    FD_points_array.append(tuple_values[0])
    FD_dollars_array.append(tuple_values[1])
    sum_points += tuple_values[0]
    counter += 1
jan_feb_data['FanDuel_Points'] = FD_points_array
jan_feb_data['FanDuel_Dollars'] = FD_dollars_array
jan_feb_data.to_csv("./OutputCSVs/Jan_Feb_statline_with_FanDuel_dollars.csv")

In [18]:
def generate_optimal_lineup(players, positions, values, costs, budget):
    num_variables = len(players)
    
    lp = LpProblem("My LP Problem", pulp.LpMaximize)
    
    d = {}
    for i in range(0, num_variables):
        d[players[i]] = LpVariable(players[i], cat="Binary")
    
    objective = sum(np.array(values) * np.array(list(d.values())))
    lp += objective
    
    pg_constraint = 0
    sg_constraint = 0
    sf_constraint = 0
    pf_constraint = 0
    c_constraint = 0
    for i in range(0, len(positions)):
        if positions[i] == "PG":
            pg_constraint += d[players[i]]
        elif positions[i] == "SG":
            sg_constraint += d[players[i]]
        elif positions[i] == "SF":
            sf_constraint += d[players[i]]
        elif positions[i] == "PF":
            pf_constraint += d[players[i]]
        else:
            c_constraint += d[players[i]]
    lp += pg_constraint == 2
    lp += sg_constraint == 2
    lp += sf_constraint == 2
    lp += pf_constraint == 2
    lp += c_constraint == 1
    
    cost = sum(np.array(costs) * np.array(list(d.values())))
    lp += cost <= budget
    
    lp.solve()
    
    lineup = [variable.name for variable in lp.variables() if variable.varValue == 1]
    print(lineup)
    #for variable in lp.variables():
    #    print ("{} = {}".format(variable.name, variable.varValue))
    #print(pulp.value(lp.objective))

In [58]:
def box_scores_for_range_of_days(start_date, end_date):
    all_tables = []
    start_month = start_date.month
    end_month = end_date.month
    for m in range(start_month, end_month + 1):
        if m == start_month:
            start_day = start_date.day
        else:
            start_day = 1
        if m == end_month:
            end_day = end_date.day
        else:
            if m == 2:
                end_day = 29
            elif m in [9, 4, 6, 11]:
                end_day = 31
            else:
                end_day = 32
        
        for d in range(start_day, end_day + 1):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            if pd.read_csv(file_name).empty:
                client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
            table = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    full_df = all_tables[0]
    for i in range(1, len(all_tables)):
        current_table = all_tables[i]
        full_df = full_df.append(current_table)
    
    full_df.index = range(full_df.shape[0])
    df = pd.read_csv("./OutputCSVs/teams_stats.csv")
    df["Team"] = df["Team"].str.upper()

    team_def = []
    team_pace = []
    team_tov = []
    opp_def = []
    opp_pace = []
    opp_tov = []
    all_games_teams = full_df[["team", "opponent"]]

    for i in range(len(all_games_teams)):
        game = all_games_teams.loc[i]
        team = game["team"].upper()
        opponent = game["opponent"].upper()
        team_def.append(df[df["Team"] == team]["DRtg"].iloc[0])
        team_pace.append(df[df["Team"] == team]["Pace"].iloc[0])
        team_tov.append(df[df["Team"] == team]["TOV%"].iloc[0])
        opp_def.append(df[df["Team"] == opponent]["DRtg"].iloc[0])
        opp_pace.append(df[df["Team"] == opponent]["Pace"].iloc[0])
        opp_tov.append(df[df["Team"] == opponent]["TOV%"].iloc[0])
    
    full_df["Team Defensive Rating"] = team_def
    full_df["Team Pace"] = team_pace
    full_df["Team Turnover %"] = team_tov
    full_df["Opponent Defensive Rating"] = opp_def
    full_df["Opponent Pace"] = opp_pace
    full_df["Opponent Turnover %"] = opp_tov
    return full_df

In [57]:
def make_predictions(start_date, end_date, output = True):
    input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals", "seconds_played", "Opponent Defensive Rating", "Opponent Turnover %"]
    full_df = box_scores_for_range_of_days(start_date, end_date)
    predicted_statlines = statline_output(statline_predictor(full_df, input_statistics))
    if output:
        output_filename = './AllCSVs/' + start_date.month + '_' + start_date.day + '_' + end_date.month + '_' + end_date.day + '_2020_box_scores.csv'
        predicted_statlines.to_csv(output_filename)
    return predicted_statlines

make_predictions(datetime(2020, 1, 1), datetime(2020, 1, 11), output = False)

Index(['slug', 'name', 'team', 'location', 'opponent', 'outcome',
       'seconds_played', 'made_field_goals', 'attempted_field_goals',
       'made_three_point_field_goals', 'attempted_three_point_field_goals',
       'made_free_throws', 'attempted_free_throws', 'offensive_rebounds',
       'defensive_rebounds', 'assists', 'steals', 'blocks', 'turnovers',
       'personal_fouls', 'game_score', 'Date'],
      dtype='object')
LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
Nikola Vučević
16
2020-01-01
Bobby Portis
17
2020-01-01
Kyle Kuzma
18
2020-01-01
Khem Birch
19
2020-01-01
Robert Covi

Marcus Morris
283
2020-01-03
Lonzo Ball
284
2020-01-03
Terrence Ross
285
2020-01-03
Derrick Favors
286
2020-01-03
LeBron James
287
2020-01-03
CJ McCollum
288
2020-01-03
Nikola Vučević
289
2020-01-03
Marcus Smart
290
2020-01-03
Jaylen Brown
291
2020-01-03
Trae Young
292
2020-01-03
Bobby Portis
293
2020-01-03
Tobias Harris
294
2020-01-03
Bam Adebayo
295
2020-01-03
Deandre Ayton
296
2020-01-03
Aron Baynes
297
2020-01-03
Joel Embiid
298
2020-01-03
Kevin Huerter
299
2020-01-03
Garrison Mathews
300
2020-01-03
Carmelo Anthony
301
2020-01-03
Enes Kanter
302
2020-01-03
Gordon Hayward
303
2020-01-03
Daniel Theis
304
2020-01-03
John Collins
305
2020-01-03
J.J. Redick
306
2020-01-03
Evan Fournier
307
2020-01-03
Aaron Gordon
308
2020-01-03
Elfrid Payton
309
2020-01-03
Ricky Rubio
310
2020-01-03
Ish Smith
311
2020-01-03
E'Twaun Moore
312
2020-01-03
Russell Westbrook
313
2020-01-03
Johnathan Williams
314
2020-01-03
Trey Burke
315
2020-01-03
Brandon Goodwin
316
2020-01-03
Markelle Fultz
317
2020-01-03

E'Twaun Moore
575
2020-01-04
Lonnie Walker
576
2020-01-04
Jaxson Hayes
577
2020-01-04
Jerami Grant
578
2020-01-04
Daniel Theis
579
2020-01-04
Dean Wade
580
2020-01-04
Malik Monk
581
2020-01-04
Gary Payton
582
2020-01-04
Derrick Walton
583
2020-01-04
Juan Hernangómez
584
2020-01-04
Jahlil Okafor
585
2020-01-04
Rayjon Tucker
586
2020-01-04
Dwayne Bacon
587
2020-01-04
Keldon Johnson
588
2020-01-04
Vlatko Čančar
589
2020-01-04
Jordan McRae
590
2020-01-04
Džanan Musa
591
2020-01-04
Tristan Thompson
592
2020-01-04
Marko Guduric
593
2020-01-04
Isaiah Thomas
594
2020-01-04
Delon Wright
595
2020-01-04
Jalen Brunson
596
2020-01-04
Matthew Dellavedova
597
2020-01-04
Ante Žižić
598
2020-01-04
Daniel Gafford
599
2020-01-04
Thon Maker
600
2020-01-04
Coby White
601
2020-01-04
Justin James
602
2020-01-04
Amile Jefferson
603
2020-01-04
Cody Martin
604
2020-01-04
Malcolm Miller
605
2020-01-04
Juwan Morgan
606
2020-01-04
Josh Reaves
607
2020-01-04
Rodney McGruder
608
2020-01-04
Boban Marjanović
609
2020-

Bryn Forbes
868
2020-01-06
Jerami Grant
869
2020-01-06
Anžejs Pasečņiks
870
2020-01-06
Terrance Ferguson
871
2020-01-06
Gary Payton
872
2020-01-06
Allen Crabbe
873
2020-01-06
Frank Jackson
874
2020-01-06
Mason Plumlee
875
2020-01-06
Harry Giles
876
2020-01-06
Nickeil Alexander-Walker
877
2020-01-06
Kris Dunn
878
2020-01-06
Brandon Goodwin
879
2020-01-06
Edmond Sumner
880
2020-01-06
Robin Lopez
881
2020-01-06
Mohamed Bamba
882
2020-01-06
Chimezie Metu
883
2020-01-06
P.J. Washington
884
2020-01-06
Mike Muscala
885
2020-01-06
Jakob Poeltl
886
2020-01-06
Justin Anderson
887
2020-01-06
Garrison Mathews
888
2020-01-06
Alen Smailagić
889
2020-01-06
Garrett Temple
890
2020-01-06
Ersan İlyasova
891
2020-01-06
Javonte Green
892
2020-01-06
DeAndre' Bembry
893
2020-01-06
Malik Monk
894
2020-01-06
Jacob Evans
895
2020-01-06
Kenrich Williams
896
2020-01-06
Caleb Swanigan
897
2020-01-06
JaKarr Sampson
898
2020-01-06
Ky Bowman
899
2020-01-06
Vince Carter
900
2020-01-06
Seth Curry
901
2020-01-06
Shaqui

Semi Ojeleye
1156
2020-01-08
Jaylen Brown
1157
2020-01-08
Malik Monk
1158
2020-01-08
Jamal Murray
1159
2020-01-08
Jordan Clarkson
1160
2020-01-08
Jakob Poeltl
1161
2020-01-08
Meyers Leonard
1162
2020-01-08
Coby White
1163
2020-01-08
Elfrid Payton
1164
2020-01-08
Alen Smailagić
1165
2020-01-08
Jayson Tatum
1166
2020-01-08
Malik Beasley
1167
2020-01-08
Dorian Finney-Smith
1168
2020-01-08
Brandon Goodwin
1169
2020-01-08
Draymond Green
1170
2020-01-08
Mitchell Robinson
1171
2020-01-08
Gary Harris
1172
2020-01-08
Matt Thomas
1173
2020-01-08
Javonte Green
1174
2020-01-08
Mohamed Bamba
1175
2020-01-08
Melvin Frazier
1176
2020-01-08
Bismack Biyombo
1177
2020-01-08
Devonte' Graham
1178
2020-01-08
T.J. McConnell
1179
2020-01-08
Kemba Walker
1180
2020-01-08
Ky Bowman
1181
2020-01-08
Isaac Bonga
1182
2020-01-08
Jordan McRae
1183
2020-01-08
Kyle Korver
1184
2020-01-08
Ignas Brazdeikis
1185
2020-01-08
Torrey Craig
1186
2020-01-08
Robin Lopez
1187
2020-01-08
Gary Payton
1188
2020-01-08
Grant Williams

Dwight Powell
1446
2020-01-10
Damyean Dotson
1447
2020-01-10
Kendrick Nunn
1448
2020-01-10
Aron Baynes
1449
2020-01-10
Miles Bridges
1450
2020-01-10
Kyle Anderson
1451
2020-01-10
Joe Harris
1452
2020-01-10
Harry Giles
1453
2020-01-10
James Johnson
1454
2020-01-10
Džanan Musa
1455
2020-01-10
Caris LeVert
1456
2020-01-10
Willie Cauley-Stein
1457
2020-01-10
Quinn Cook
1458
2020-01-10
Jalen Brunson
1459
2020-01-10
T.J. McConnell
1460
2020-01-10
Frank Ntilikina
1461
2020-01-10
Thaddeus Young
1462
2020-01-10
Dwayne Bacon
1463
2020-01-10
Derrick Jones
1464
2020-01-10
Brook Lopez
1465
2020-01-10
Tony Bradley
1466
2020-01-10
Robin Lopez
1467
2020-01-10
Mikal Bridges
1468
2020-01-10
Dario Šarić
1469
2020-01-10
Avery Bradley
1470
2020-01-10
Jacob Evans
1471
2020-01-10
Ivica Zubac
1472
2020-01-10
Amile Jefferson
1473
2020-01-10
Yogi Ferrell
1474
2020-01-10
Duncan Robinson
1475
2020-01-10
Jared Dudley
1476
2020-01-10
Anžejs Pasečņiks
1477
2020-01-10
Thanasis Antetokounmpo
1478
2020-01-10
DeAndre' B

2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02
2020-01-02

2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-07
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08
2020-01-08

,name,team,date,location,opponent,minutes,made_two_point_field_goals,made_three_point_field_goals,made_free_throws,rebounds,assists,blocks,steals,turnovers,recent_average
0,LeBron James,LOS ANGELES LAKERS,2020-01-01,HOME,PHOENIX SUNS,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0
1,Giannis Antetokounmpo,MILWAUKEE BUCKS,2020-01-01,HOME,MINNESOTA TIMBERWOLVES,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0
2,D.J. Augustin,ORLANDO MAGIC,2020-01-01,AWAY,WASHINGTON WIZARDS,0,0,0.766,1.373,3.495,2.06,0.329,0,0.329,0
3,Kelly Oubre,PHOENIX SUNS,2020-01-01,AWAY,LOS ANGELES LAKERS,0,0,0.766,1.373,3.495,2.06,0.329,0,0.329,0
4,Mitchell Robinson,NEW YORK KNICKS,2020-01-01,HOME,PORTLAND TRAIL BLAZERS,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0
5,Anthony Davis,LOS ANGELES LAKERS,2020-01-01,HOME,PHOENIX SUNS,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0
6,Hassan Whiteside,PORTLAND TRAIL BLAZERS,2020-01-01,AWAY,NEW YORK KNICKS,0,0,0.766,1.373,3.495,2.06,0.329,0,0.329,0
7,Devin Booker,PHOENIX SUNS,2020-01-01,AWAY,LOS ANGELES LAKERS,0,0,0.766,1.373,3.495,2.06,0.329,0,0.329,0
8,Julius Randle,NEW YORK KNICKS,2020-01-01,HOME,PORTLAND TRAIL BLAZERS,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0
9,Bradley Beal,WASHINGTON WIZARDS,2020-01-01,HOME,ORLANDO MAGIC,0,0,0.766,1.0441,3.495,2.06,0.329,0,0.329,0


In [74]:
import pytz

def predict_next_day(start_date, game_date):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    team_objs_on_date = [game["home_team"] for game in schedule_on_date] + [game["away_team"] for game in schedule_on_date]
    teams_on_date = [t.name.replace("_", " ") for t in team_objs_on_date]
    predictions_for_date = make_predictions(start_date, game_date, output = False)
    players_on_date = predictions_for_date[predictions_for_date.team.isin(teams_on_date)]
    mstr, dstr = str(game_date.month), str(game_date.day)
    if game_date.month < 10:
        mstr = "0" + mstr
    if game_date.day < 10:
        dstr = "0" + dstr
    tstr = mstr + "-" + dstr
    statlines_on_date = players_on_date[players_on_date.date.str.contains(tstr)]
    statlines_on_date.index = range(statlines_on_date.shape[0])
    output_filename = './AllCSVs/predictions_for_' + mstr + "_" + dstr + '_from_' + str(start_date.month) + '_' + str(start_date.day) + '_2020.csv'
    statlines_on_date.to_csv(output_filename)
    return statlines_on_date
predict_next_day(datetime(2020, 1, 1), datetime(2020, 1, 12))

LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
Nikola Vučević
16
2020-01-01
Bobby Portis
17
2020-01-01
Kyle Kuzma
18
2020-01-01
Khem Birch
19
2020-01-01
Robert Covington
20
2020-01-01
Marcus Morris
21
2020-01-01
Terrence Ross
22
2020-01-01
CJ McCollum
23
2020-01-01
Donte DiVincenzo
24
2020-01-01
Jordan McRae
25
2020-01-01
Troy Brown
26
2020-01-01
Josh Okogie
27
2020-01-01
Ricky Rubio
28
2020-01-01
Wesley Iwundu
29
2020-01-01
Ish Smith
30
2020-01-01
Gorgui Dieng
31
2020-01-01
Brook Lopez
32
2020-01-01
Eric Bledsoe
33
2020-01-01
Evan Fournier
34
2020-01-01
Ersan İlyasova
35
2020-01-01
Gar

Deandre Ayton
296
2020-01-03
Aron Baynes
297
2020-01-03
Joel Embiid
298
2020-01-03
Kevin Huerter
299
2020-01-03
Garrison Mathews
300
2020-01-03
Carmelo Anthony
301
2020-01-03
Enes Kanter
302
2020-01-03
Gordon Hayward
303
2020-01-03
Daniel Theis
304
2020-01-03
John Collins
305
2020-01-03
J.J. Redick
306
2020-01-03
Evan Fournier
307
2020-01-03
Aaron Gordon
308
2020-01-03
Elfrid Payton
309
2020-01-03
Ricky Rubio
310
2020-01-03
Ish Smith
311
2020-01-03
E'Twaun Moore
312
2020-01-03
Russell Westbrook
313
2020-01-03
Johnathan Williams
314
2020-01-03
Trey Burke
315
2020-01-03
Brandon Goodwin
316
2020-01-03
Markelle Fultz
317
2020-01-03
Mitchell Robinson
318
2020-01-03
Brandon Ingram
319
2020-01-03
Jrue Holiday
320
2020-01-03
Julius Randle
321
2020-01-03
Alex Len
322
2020-01-03
JaVale McGee
323
2020-01-03
D.J. Augustin
324
2020-01-03
Al Horford
325
2020-01-03
Mohamed Bamba
326
2020-01-03
Jaxson Hayes
327
2020-01-03
Furkan Korkmaz
328
2020-01-03
Allen Crabbe
329
2020-01-03
Jayson Tatum
330
2020-

Džanan Musa
591
2020-01-04
Tristan Thompson
592
2020-01-04
Marko Guduric
593
2020-01-04
Isaiah Thomas
594
2020-01-04
Delon Wright
595
2020-01-04
Jalen Brunson
596
2020-01-04
Matthew Dellavedova
597
2020-01-04
Ante Žižić
598
2020-01-04
Daniel Gafford
599
2020-01-04
Thon Maker
600
2020-01-04
Coby White
601
2020-01-04
Justin James
602
2020-01-04
Amile Jefferson
603
2020-01-04
Cody Martin
604
2020-01-04
Malcolm Miller
605
2020-01-04
Juwan Morgan
606
2020-01-04
Josh Reaves
607
2020-01-04
Rodney McGruder
608
2020-01-04
Boban Marjanović
609
2020-01-04
Semi Ojeleye
610
2020-01-04
Theo Pinson
611
2020-01-04
Edmond Sumner
612
2020-01-04
D.J. Wilson
613
2020-01-04
Torrey Craig
614
2020-01-04
Solomon Hill
615
2020-01-04
Thanasis Antetokounmpo
616
2020-01-04
Sterling Brown
617
2020-01-04
Josh Magette
618
2020-01-04
Kenrich Williams
619
2020-01-04
Yogi Ferrell
620
2020-01-04
Patrick Patterson
621
2020-01-04
Kyle Anderson
622
2020-01-04
Glenn Robinson
623
2020-01-04
Deonte Burton
624
2020-01-04
Stanl

Jakob Poeltl
886
2020-01-06
Justin Anderson
887
2020-01-06
Garrison Mathews
888
2020-01-06
Alen Smailagić
889
2020-01-06
Garrett Temple
890
2020-01-06
Ersan İlyasova
891
2020-01-06
Javonte Green
892
2020-01-06
DeAndre' Bembry
893
2020-01-06
Malik Monk
894
2020-01-06
Jacob Evans
895
2020-01-06
Kenrich Williams
896
2020-01-06
Caleb Swanigan
897
2020-01-06
JaKarr Sampson
898
2020-01-06
Ky Bowman
899
2020-01-06
Vince Carter
900
2020-01-06
Seth Curry
901
2020-01-06
Shaquille Harrison
902
2020-01-06
Paul Millsap
903
2020-01-06
Bismack Biyombo
904
2020-01-06
Thanasis Antetokounmpo
905
2020-01-06
DeMarre Carroll
906
2020-01-06
James Ennis
907
2020-01-06
Chandler Hutchison
908
2020-01-06
Goga Bitadze
909
2020-01-06
Carsen Edwards
910
2020-01-06
Bruno Fernando
911
2020-01-06
Hamidou Diallo
912
2020-01-06
Darius Bazley
913
2020-01-06
Nicolas Claxton
914
2020-01-06
Semi Ojeleye
915
2020-01-06
D.J. Wilson
916
2020-01-06
Wenyen Gabriel
917
2020-01-06
Admiral Schofield
918
2020-01-06
Justin James
919

Devonte' Graham
1178
2020-01-08
T.J. McConnell
1179
2020-01-08
Kemba Walker
1180
2020-01-08
Ky Bowman
1181
2020-01-08
Isaac Bonga
1182
2020-01-08
Jordan McRae
1183
2020-01-08
Kyle Korver
1184
2020-01-08
Ignas Brazdeikis
1185
2020-01-08
Torrey Craig
1186
2020-01-08
Robin Lopez
1187
2020-01-08
Gary Payton
1188
2020-01-08
Grant Williams
1189
2020-01-08
Chris Boucher
1190
2020-01-08
JaKarr Sampson
1191
2020-01-08
Tacko Fall
1192
2020-01-08
Ish Smith
1193
2020-01-08
Luke Kornet
1194
2020-01-08
E'Twaun Moore
1195
2020-01-08
Ryan Arcidiacono
1196
2020-01-08
Kadeem Allen
1197
2020-01-08
RJ Barrett
1198
2020-01-08
Shaquille Harrison
1199
2020-01-08
Terrence Ross
1200
2020-01-08
Goga Bitadze
1201
2020-01-08
Carsen Edwards
1202
2020-01-08
Danuel House
1203
2020-01-08
Enes Kanter
1204
2020-01-08
Juan Hernangómez
1205
2020-01-08
B.J. Johnson
1206
2020-01-08
Ed Davis
1207
2020-01-08
Cam Reddish
1208
2020-01-08
Nigel Williams-Goss
1209
2020-01-08
Wesley Matthews
1210
2020-01-08
Ian Mahinmi
1211
2020-

Dwayne Bacon
1463
2020-01-10
Derrick Jones
1464
2020-01-10
Brook Lopez
1465
2020-01-10
Tony Bradley
1466
2020-01-10
Robin Lopez
1467
2020-01-10
Mikal Bridges
1468
2020-01-10
Dario Šarić
1469
2020-01-10
Avery Bradley
1470
2020-01-10
Jacob Evans
1471
2020-01-10
Ivica Zubac
1472
2020-01-10
Amile Jefferson
1473
2020-01-10
Yogi Ferrell
1474
2020-01-10
Duncan Robinson
1475
2020-01-10
Jared Dudley
1476
2020-01-10
Anžejs Pasečņiks
1477
2020-01-10
Thanasis Antetokounmpo
1478
2020-01-10
DeAndre' Bembry
1479
2020-01-10
Goga Bitadze
1480
2020-01-10
Jahlil Okafor
1481
2020-01-10
Delon Wright
1482
2020-01-10
Daniel Gafford
1483
2020-01-10
Tyus Jones
1484
2020-01-10
Rodney McGruder
1485
2020-01-10
Jae Crowder
1486
2020-01-10
Wenyen Gabriel
1487
2020-01-10
Alen Smailagić
1488
2020-01-10
JaMychal Green
1489
2020-01-10
Derrick Favors
1490
2020-01-10
Danny Green
1491
2020-01-10
Solomon Hill
1492
2020-01-10
Cameron Johnson
1493
2020-01-10
Maxi Kleber
1494
2020-01-10
Antonius Cleveland
1495
2020-01-10
Drew

Vince Carter
1769
2020-01-12
Aron Baynes
1770
2020-01-12
Patrick Beverley
1771
2020-01-12
Alex Len
1772
2020-01-12
Isaac Bonga
1773
2020-01-12
Jordan Poole
1774
2020-01-12
Ish Smith
1775
2020-01-12
Marco Belinelli
1776
2020-01-12
Meyers Leonard
1777
2020-01-12
Mikal Bridges
1778
2020-01-12
Thomas Bryant
1779
2020-01-12
Duncan Robinson
1780
2020-01-12
Elie Okobo
1781
2020-01-12
Pascal Siakam
1782
2020-01-12
Glenn Robinson
1783
2020-01-12
Rodions Kurucs
1784
2020-01-12
Chris Boucher
1785
2020-01-12
Ricky Rubio
1786
2020-01-12
Draymond Green
1787
2020-01-12
Rondae Hollis-Jefferson
1788
2020-01-12
Jakob Poeltl
1789
2020-01-12
Isaiah Thomas
1790
2020-01-12
Kevin Huerter
1791
2020-01-12
Mason Plumlee
1792
2020-01-12
Spencer Dinwiddie
1793
2020-01-12
Kyle Anderson
1794
2020-01-12
Maurice Harkless
1795
2020-01-12
Dario Šarić
1796
2020-01-12
Garrett Temple
1797
2020-01-12
Devin Booker
1798
2020-01-12
Bobby Portis
1799
2020-01-12
Damian Jones
1800
2020-01-12
Georges Niang
1801
2020-01-12
Jae Cro

,name,team,date,location,opponent,minutes,made_two_point_field_goals,made_three_point_field_goals,made_free_throws,rebounds,assists,blocks,steals,turnovers,recent_average
0,Jonas Valančiūnas,MEMPHIS GRIZZLIES,2020-01-12,HOME,GOLDEN STATE WARRIORS,26.3597,4.60373,1.3331,2.90537,7.13024,3.47788,0.92732,0.75132,0.92732,35.349
1,Jimmy Butler,MIAMI HEAT,2020-01-12,AWAY,NEW YORK KNICKS,33.1383,5.84956,1.00354,5.7289,6.86564,4.86316,0.4026,1.19727,0.4026,42.949
2,D'Angelo Russell,GOLDEN STATE WARRIORS,2020-01-12,AWAY,MEMPHIS GRIZZLIES,0,0,0.766,1.373,3.495,2.06,0.329,0,0.329,0
3,RJ Barrett,NEW YORK KNICKS,2020-01-12,HOME,MIAMI HEAT,31.3877,5.43019,1.37376,4.20174,5.65782,2.4896,0.42808,1.17087,0.42808,25.16
4,DeMar DeRozan,SAN ANTONIO SPURS,2020-01-12,AWAY,TORONTO RAPTORS,35.1935,7.68768,0.99926,6.4425,5.79354,5.29988,0.3387,0.8601,0.3387,46.035
5,Kelly Oubre,PHOENIX SUNS,2020-01-12,HOME,CHARLOTTE HORNETS,36.737,3.12964,2.26186,3.46068,6.24585,2.63192,0.47096,0.98664,0.47096,36.1
6,Kawhi Leonard,LOS ANGELES CLIPPERS,2020-01-12,AWAY,DENVER NUGGETS,33.3487,6.60699,2.0393,3.9664,6.42409,4.326,0.53326,1.50897,0.53326,48.618
7,Ian Mahinmi,WASHINGTON WIZARDS,2020-01-12,HOME,UTAH JAZZ,21.1032,3.89777,0.94576,2.16491,6.95198,2.9506,1.17504,1.04358,1.17504,20.587
8,Montrezl Harrell,LOS ANGELES CLIPPERS,2020-01-12,AWAY,DENVER NUGGETS,30.5173,7.67573,0.80238,5.99251,6.56966,3.32432,0.69296,0.68424,0.69296,35.702
9,Rudy Gobert,UTAH JAZZ,2020-01-12,AWAY,WASHINGTON WIZARDS,30.7592,5.61211,0.766,4.03526,10.1396,3.33072,1.37584,0.83946,1.37584,42.828


In [28]:
def append_actual_results(start_date, game_date):
    predicted_results = predict_next_day(start_date, game_date)
    actual_results = box_scores_for_range_of_days(game_date, game_date)
    pred_outputs = [get_points(actual_results["name" == player_name]) for player_name in predicted_results["name"]]
    predicted_results["actual"] = pred_outputs
    output_filename = './AllCSVs/' + game_date.month + '_' + game_date.day + '_2020_box_scores_predicted_with_actual.csv'
    predicted_results.to_csv(output_filename)
    return predicted_results

In [29]:
def alternate_contests(start_date, end_date, game_date, search_datetime, inclusive):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    times_on_date = np.unique([game['start_time'] - timedelta(hours = 4) for game in schedule_on_date])
    if not inclusive:
        game_teams = [game["home_team"] for game in schedule_on_date if game['start_time'] == search_datetime] + [game["away_team"] for game in schedule_on_date if game['start_time'] == search_datetime]
    else:
        times_after_search = [t for t in times_on_date if 60 * t.hour + t.minute >= 60 * search_datetime.hour + search_datetime.minute]
        game_teams = [game["home_team"] for game in schedule_on_date if game['start_time'] in times_after_search] + [game["away_team"] for game in schedule_on_date if game['start_time'] in times_after_search]
    predictions_for_date = make_predictions(start_date, end_date, output = False)
    players_on_date = [statline for statline in predictions_for_date if statline["team"] in teams_on_date]
    output_filename = './AllCSVs/' + game_date.month + '_' + game_date.day + '_2020_alternate_' + search_datetime.hour + "_" + search_datetime.minute + '.csv'
    players_on_date.to_csv(output_filename)
    return players_on_date
alternate_contests(datetime(2020, 1, 1), datetime(2020, 1, 2), datetime(2020, 1, 3), datetime(2020, 1, 2, 21, 30), True)

LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
Nikola Vučević
16
2020-01-01
Bobby Portis
17
2020-01-01
Kyle Kuzma
18
2020-01-01
Khem Birch
19
2020-01-01
Robert Covington
20
2020-01-01
Marcus Morris
21
2020-01-01
Terrence Ross
22
2020-01-01
CJ McCollum
23
2020-01-01
Donte DiVincenzo
24
2020-01-01
Jordan McRae
25
2020-01-01
Troy Brown
26
2020-01-01
Josh Okogie
27
2020-01-01
Ricky Rubio
28
2020-01-01
Wesley Iwundu
29
2020-01-01
Ish Smith
30
2020-01-01
Gorgui Dieng
31
2020-01-01
Brook Lopez
32
2020-01-01
Eric Bledsoe
33
2020-01-01
Evan Fournier
34
2020-01-01
Ersan İlyasova
35
2020-01-01
Gar

TypeError: string indices must be integers

In [ ]:
def optimal_lineup_games(input_games, ):

In [30]:
all_abbrv = {'ATL':'Atlanta', 'BOS':'Boston', 'BKN':'Brooklyn', 'CHA':'Charlotte', 'CHI':'Chicago', 'CLE':'Cleveland', 'DAL':'Dallas',
            'DEN':'Denver', 'DET':'Detroit', 'GSW':'Golden State', 'HOU':'Houston', 'IND':'Indiana', 'LAC':'LA Clippers', 'LAL':'LA Lakers',
            'MEM':'Memphis', 'MIA':'Miami', 'MIL':'Milwaukee', 'MIN':'Minnesota', 'NOR':'New Orleans', 'NYK':'New York', 'OKC':'Oklahoma City', 'ORL':'Orlando',
            'PHI':'Philadelphia', 'PHO':'Phoenix', 'POR':'Portland', 'SAC':'Sacramento', 'SAS':'San Antonio', 'TOR':'Toronto', 'UTA':'Utah', 'WAS':'Washington'}
dkp = pd.read_csv('./OutputCSVs/draftkings_points.csv')
dkp = dkp[pd.isna(dkp.position)==False]
dbp = pd.read_csv('./OutputCSVs/team_def_vs_position.csv')

ranks = []
seen = {}
player_names = [seen.setdefault(x, x) for x in dkp['name'] if x not in seen]
abbrv = [str(dkp.loc[dkp['name']==x, 'team'].iloc[0]).upper() for x in player_names]
player_positions = [dkp.loc[dkp['name']==x, 'position'].iloc[0] for x in player_names]
team_names = [all_abbrv.get(x) for x in abbrv]

for i in range(0, len(player_names)):
    if len(player_positions[i])<=2:
        ranks.append(dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i])].iloc[0])
    else:
        first_subrank =  dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i][:2])].iloc[0]
        second_subrank =  dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i][3:])].iloc[0]
        ranks.append((first_subrank + second_subrank)/2)

outputDefense = pd.DataFrame({'name':player_names, 'pos':player_positions, 'team':team_names, 'rank':ranks})
outputDefense.to_csv('./OutputCSVs/player_def_vs_position.csv', header=True, index=False)